# RNN

## RNN Básica

### Arquitectura

La arquitectura de una red neuronal recurrente (RNN) básica puede definirse como una función:

$f_{\theta} =  (x_t, h_t) \rightarrow (y_t, h_{t+1})$

Donde:
- $x_t$ es el vector de entrada.
- $h_t$ es el vector de estado oculto.
- $y_t$ es el vector de salida.
- $\theta$ son los parámetros de la red.

Este tipo de red neuronal mapea un input $x_t$ a un output $y_t$, donde $h_t$ hace la función de "memoria" de la red neuronal. Este estado oculto $h_t$ se actualiza en cada paso de tiempo tomando como referencia el estado oculto previo. 

Si analizamos la arquitectura paso a paso encontraremos:

1. Capa de entrada:
    - Representa los datos secuenciales
    - Cada paso temporal tendrá su propio vector de entrada

2. Capa oculta (recurrente):
    - Procesa el input actual y el estado oculto previo.
    - $h_t = tanh(W_h x(t) + U_h h(t-1) + b_h)$
        - h(t) es el estado actual oculto
        - x(t) es la entrada/input actual
        - h(t-1) es el estado oculto previo
        - W_h es la matriz de pesos input-oculta
        - U_h es la matriz de pesos oculta-oculta
        - b_h es el sesgo de la capa oculta
3. Capa de salida:
    - Genera las predicciones según el estado oculto.
    - $y(t) = softmax(W_yh(t)+b_y)$
        - W_y es la matriz de pesos oculta-salida
        - b_y es el sesgo de salida
        - softmax es la función de normalización

Los pesos de las matrices van a ser los mismos en todos los pasos temporales, permitiendo de esta forma procesar secuencias de distinta longitud, y permitiendo el aprendizaje de patrones independientemente de la posición en la secuencia.

Las funciones de activación son a modo de guía, también puede utilizarse ReLU como normalización.

Los mecanismos de entrenamiento son similares a los que ya conocemos:

- Backpropagation Through Time
- Computación de gradientes en todos los pasos temporales
- Pasos:
    1. Secuencia forward
    2. Computación de la pérdida
    3. Error de retropropagación
    4. Actualización de pesos

## Limitaciones

### Vanishing gradient

Durante la retropropagación los gradientes se calculan mediante la regla de la cadena, multiplicando las derivadas parciales en cada capa.

Para funciones de activación con derivadas menores a 1, las multiplicaciones repetitivas provocan un decrecimiento exponencial que puede provocar que el gradiente sea extremadamente pequeño. 

Esto provoca que las redes no puedan aprender dependencias de largo plazo, actualizaciones mínimas de los pesos, y por tanto los modelos se vuelven ineficientes.

### Exploding gradient

El proceso opuesto al vanishing gradient, cuando las funciones de activación tienen derivadas mayores a 1 los gradientes crecen de forma exponencial. 

Esto provoca que las actualizaciones de pesos sean excesivamente grandes, y estas actualizaciones se vuelvan inestables, lo que provoca divergencia en el modelo y configuraciones sin sentido, por lo que no puede entrenarse el modelo.

Existen varias soluciones tanto de inicialización de los pesos (matrices ortogonales, estrategias de normalización, etc.) como de optimización (ADAM, RMSprop). La recomendación más prevalente es utilizar arquitecturas más sólidas como LSTM y GRUs.

### Vanilla RNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definir el modelo RNN
class VanillaRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Inicializar el estado oculto con ceros
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        # Pasar los datos a través de la capa RNN
        out, _ = self.rnn(x, h0)
        # Pasar la salida de la última capa RNN a través de una capa totalmente conectada
        out = self.fc(out[:, -1, :])
        return out

# Hiperparámetros
input_size = 10  # Tamaño de la entrada
hidden_size = 20  # Tamaño del estado oculto
output_size = 1  # Tamaño de la salida
num_epochs = 100  # Número de épocas de entrenamiento
learning_rate = 0.001  # Tasa de aprendizaje

# Generar algunos datos aleatorios
x_train = torch.randn(100, 5, input_size)  # Datos de entrada de entrenamiento
y_train = torch.randn(100, output_size)  # Datos de salida de entrenamiento

# Inicializar el modelo, la función de pérdida y el optimizador
model = VanillaRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # Función de pérdida de error cuadrático medio
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Optimizador Adam

# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()  # Poner el modelo en modo de entrenamiento
    outputs = model(x_train)  # Obtener las predicciones del modelo
    loss = criterion(outputs, y_train)  # Calcular la pérdida
    
    optimizer.zero_grad()  # Limpiar los gradientes
    loss.backward()  # Calcular los gradientes
    optimizer.step()  # Actualizar los parámetros del modelo
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')  # Imprimir la pérdida cada 10 épocas

print("Entrenamiento completo.")  # Indicar que el entrenamiento ha terminado

## LSTM

Las redes neuronales LSTM (Long short-term memory) están diseñadas para solucionar un problema presente en las RNN "normales", el problema de las dependencias a largo plazo.

Este problema parte de cuando existe demasiada distancia entre el paso temporal actual y el paso temporal del que necesitamos el contexto. 

Por ejemplo, para predecir "español" en la frase "crecí en España, hablo de forma fluida el \it{español}", se requiere la información del paso temporal que incluye la palabra "España".

Si esta distancia es demasiado grande, la RNN tradicional no será capaz de hacer la conexión para esta información.

Además, serán útiles para solventar el problema de gradientes explicados previamente.

### Arquitectura

La arquitectura de una red LSTM consta de una serie de puertas que regulan la información.

Su punto central es la célula de memoria, que es controlada por tres puertas que regulan la información.

- Puertas (redes neurales sigmoides):
    - Forget gate: Decide la información a eliminar
        - $f(t) = \sigma (W_f [h_{t-1}, x_t] + b_f)$
    - Input gate: Determina la información a almacenar
        - $i(t) = \sigma (W_i [h_{t-1}, x_t] + b_i)$
    - Output gate: Controla que información de la célula de memoria es output.
        - $o(t) = \sigma (W_o [h_{t-1}, x_t] + b_o)$
    - Candidate memory:
        - Genera los valores que pueden añadirse a la célula.
        - $C = tanh(W_C [h_{t-1}, x_t] + b_C)$
- Célula de memoria:
    - Mantiene y modifica la información a largo plazo
    - Permite la preservación de información selctiva.
    - Persiste entre los pasos temporales con interacciones mínimas.


### Data flow:

1. A cada paso temporal, el input se concatena con el estado oculto del paso temporal previo. 
2. La forget gate determina que partes del estado de la célula (t-1) se eliminan. 
3. La input gate determina que información va a guardarse en la célula
4. Se actualiza el estado de la célula combinando la información antigua con la nueva información candidata.
5. La output gate determina que partes de la célula actualizada se usan para computan el estado actual oculto, el cual se pasa al siguiente paso temporal.


### Limitaciones

Estas redes son costosas a nivel de computación debido a su arquitectura de puertas y parametrización, lo que las puede hacer lentas, además de requerir bastantes recursos debido a la cantidad de multiplicaciones matriciales que incluye.

Son además sensibles a overfitting debido a su alto número de parámetros, por lo que son muy sensibles a una elección correcta de hiperparámetros. 

Al ser puramente secuenciales, son bastante limitadas para computación paralela, por lo que pierden con respecto a modelos de Transformer.

In [ ]:
# Definir el modelo LSTM
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        # La capa LSTM procesa secuencias de entrada
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        # Capa lineal para transformar la salida del LSTM al tamaño deseado
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Inicializar el estado oculto (h0) y el estado de la celda (c0) con ceros
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        
        # Procesar la secuencia a través del LSTM
        # out contiene las salidas para cada paso de tiempo
        # (hn, cn) son los estados finales del LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        # Usar solo la última salida de la secuencia
        out = self.fc(out[:, -1, :])
        return out

# Crear y entrenar el modelo LSTM con los mismos hiperparámetros
model_lstm = LSTM(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=learning_rate)

# Bucle de entrenamiento
for epoch in range(num_epochs):
    model_lstm.train()  # Activar modo entrenamiento
    outputs = model_lstm(x_train)  # Forward pass
    loss = criterion(outputs, y_train)  # Calcular pérdida
    
    # Backpropagation y optimización
    optimizer.zero_grad()  # Limpiar gradientes anteriores
    loss.backward()  # Calcular gradientes
    optimizer.step()  # Actualizar pesos
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Entrenamiento del LSTM completo.")

## Gated Recurrent Unit

Estas redes neuronales son una variente simplificada de las LSTM, con una arquitectura más sencilla que reduce costes de computacíón.

### Arquitectura

Esta variante reemplaza las tres puertas y celula de memoria de la LSTM con dos puertas, las cuales regulan el flujo de información para el estado oculto.

- Hidden state ($h_t$): Es el centro de memoria de la red, en este caso no es una célula separada como en LSTM.
- Reset gate ($r_t$): Determina que debe olvidarse del estado oculto previo.
    - $r_t = \sigma(W_r [h_{t-1}, x_t] + b_r)$
- Update gate ($z_t$): Determina que debe mantenerse del estado oculto previo, y que debe actualizarse con nueva información.
    - $z_t = \sigma(W_r [h_{t-1}, x_t] + b_z)$

### Data Flow

1. A cada paso temporal se toma el input actual y el estado oculto del paso previo.
2. La reset gate determina que partes se deben ignorar del estado oculto previo, cuanto más cerca de 0 son los valores, menor influencia del estado oculto.
3. La update gate determina cuanta información nueva debe incorporar y cuanta mantener del estado previo.
4. Se compone un nuevo candidato a estado oculto con la reset gate.
5. La update gate combina el candidato a estado oculto y el estado oculto previo para generar el nuevo estado oculto.

### Limitaciones

Al ser menos compleja que la arquitectura LSTM, esta red tendrá más problemas en tareas de mucha complejidad. Además, la reducción de puertas permite menos control sobre la retención de memoria. 

Al igual que LSTM son muy sensibles a los hiperparametros para conseguir un rendimiento adecuado.

In [ ]:
# Definir el modelo GRU
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        # La capa GRU procesa secuencias de entrada
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        # Capa lineal para transformar la salida del GRU al tamaño deseado
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Inicializar el estado oculto (h0) con ceros
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        
        # Procesar la secuencia a través del GRU
        # out contiene las salidas para cada paso de tiempo
        # hn es el estado final del GRU
        out, hn = self.gru(x, h0)
        
        # Usar solo la última salida de la secuencia
        out = self.fc(out[:, -1, :])
        return out

# Crear y entrenar el modelo GRU con los mismos hiperparámetros
model_gru = GRU(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_gru.parameters(), lr=learning_rate)

# Bucle de entrenamiento
for epoch in range(num_epochs):
    model_gru.train()  # Activar modo entrenamiento
    outputs = model_gru(x_train)  # Forward pass
    loss = criterion(outputs, y_train)  # Calcular pérdida
    
    # Backpropagation y optimización
    optimizer.zero_grad()  # Limpiar gradientes anteriores
    loss.backward()  # Calcular gradientes
    optimizer.step()  # Actualizar pesos
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Entrenamiento del GRU completo.")